# Segmenting and Clustering Neighborhoods in Toronto

## Part 1

Create and clean a *pandas* dataframe using the data provided on Wikipedia's table of Canadian postal codes.

In [1]:
# install appropriate packages
!pip install beautifulsoup4 # install the BeautifulSoup library for web scraping
!pip install lxml # install lxml parser to break down the html page into parts
!pip install requests # install the requests library
print('...Packages installed!')

...Packages installed!


In [2]:
# import necessary packages
from bs4 import BeautifulSoup # bs4 = beautifulsoup4
import requests
import pandas as pd

#### Use BeautifulSoup to create the dataframe

In [3]:
# pull code from the web via the requests library
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
# use BeautifulSoup to scrape and lxml to parse
soup = BeautifulSoup(source, 'lxml')

In [5]:
# prettify method indents the html code, making it easier to read code within each tag
# commenting this line of code out because don't know how to hide the lengthy code blob on GitHub
## print(soup.prettify()) 

In [6]:
# retrieve only the text within the <table> tag, then read the data into a pandas dataframe
table = soup.find_all('table')
df = pd.read_html(str(table))[0] # [0] converts data from a list to a dataframe, remove it to see!
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


#### Clean the dataframe

In [7]:
# rename headers
labels = ['PostalCode', 'Borough', 'Neighborhood']
df.columns = labels

In [8]:
import numpy as np

# replace "Not assigned" values in the Borough column to NaN for easy removal
df['Borough'].replace("Not assigned", np.nan, inplace = True)
df

,PostalCode,Borough,Neighborhood
0,M1A,NaN,Not assigned
1,M2A,NaN,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


In [9]:
# remove rows with NaN values in the Borough column
df.dropna(subset=["Borough"], axis=0, inplace=True)
df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
...,...,...,...
282,M8Z,Etobicoke,Kingsway Park South West
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West


In [10]:
# check frequency of neighborhoods in each PostalCode
df['PostalCode'].value_counts()

M8Y    8
M9V    8
M5V    7
M9B    5
M8Z    5
      ..
M1X    1
M5E    1
M4M    1
M4P    1
M9A    1
Name: PostalCode, Length: 103, dtype: int64

In [11]:
# join neighborhoods that exist within the same PostalCode and Borough into 1 row each
# use commas as seperator
df = df.groupby(['PostalCode','Borough']).agg(lambda x: ', '.join(x))
df

## length of dataframe was 103 with the above value_counts() check, so after joining should have 103 rows/different PostalCodes

,,Neighborhood
PostalCode,Borough,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."


In [12]:
# reset index
df.reset_index(inplace=True)
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [13]:
# check if 'Not assigned' values exist in the Neighborhood column
np.where(df['Neighborhood'] == 'Not assigned')

(array([85]),)

In [14]:
# examine row 85
df.iloc[85]

PostalCode               M7A
Borough         Queen's Park
Neighborhood    Not assigned
Name: 85, dtype: object

In [15]:
# change row 85's Neighborhood to Queen's Park (so it matches its' Borough)
df.loc[85, 'Neighborhood'] = "Queen's Park"

In [16]:
# check for any more 'Not assigned' values
np.where(df['Neighborhood'] == 'Not assigned')

(array([], dtype=int64),)

In [17]:
df.shape

(103, 3)

*pandas* dataframe is now clean!

---

## Part 2

Find the latitude and longitude coordinates of each postal code.

In [18]:
#install geocoder package for retrieving lat and long coordinates
!pip install geocoder
print('... geocoder installed!')

... geocoder installed!


In [19]:
import geocoder

#### Use geocoder and a for loop to discover coordinates

In [20]:
latitude = [] # brackets represent a blank list (will put result into these)
longitude = []

# create a for loop to retrieve coordinates for each PostalCode
for code in df['PostalCode']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng
    #print(code, lat_lng_coords) # commenting out so won't see long wall of code on GitHub - uncomment to ensure correct results
    
    latitude.append(lat_lng_coords[0]) # use append and [0] to add first index of lat_lng_coords to the latitude variable
    longitude.append(lat_lng_coords[1])
    
## wait a minute for the loop to cycle through

In [21]:
# add a Latitude and Longitude column to the dataframe - populate with latitude and longitude lists
df['Latitude'] = latitude
df['Longitude'] = longitude
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
...,...,...,...,...,...
98,M9N,York,Weston,43.704855,-79.517552
99,M9P,Etobicoke,Westmount,43.696300,-79.530399
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.686915,-79.557276
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.743205,-79.584701


Latitude and longitude coordinates have been found!

---

## Part 3

Explore and cluster Toronto neighborhoods then create a map to visualize these clusters.

#### Create a Toronto map plotting the boroughs and neighborhoods of your data

In [22]:
# find Toronto's latitude and longitude coordinates
g = geocoder.arcgis('Toronto, Ontario')
g.latlng

latitude = g.latlng[0]
longitude = g.latlng[1]

In [23]:
import folium # map rendering library

# create a map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add neighborhood(s), borough markers
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Create a new dataframe and map of only "Toronto" boroughs

In [24]:
# see what boroughs are in the dataset
df['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [25]:
# create a new dataframe of only boroughs that contain the word Toronto
df_toronto = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676845,-79.295225
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314673
3,M4M,East Toronto,Studio District,43.662766,-79.334830
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085


In [26]:
df_toronto.shape

(38, 5)

So only 38 boroughs out of the 103 contain the word Toronto.

Now create a map of only boroughs containing the word Toronto.

In [27]:
# create a map of Toronto
map_toronto_bor = folium.Map(location=[latitude, longitude], zoom_start=11)

# add neighborhood(s), borough markers
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_bor)  
    
map_toronto_bor

#### Use Foursquare to retrieve nearby venue data, and put this data into a *toronto_venues* dataframe

In [28]:
# write down your Foursquare credentials and version
CLIENT_ID = 'OTTSMWX5Y1XYDLR3OWHUWVEVDTH1HO3N2BKCTJD4JMGNZYIH' # your Foursquare ID
CLIENT_SECRET = 'TTST1GFSD4HKPE4VQAWJB3YVLDGZ2VIZTBI03BZQTXFI5FYA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: OTTSMWX5Y1XYDLR3OWHUWVEVDTH1HO3N2BKCTJD4JMGNZYIH
CLIENT_SECRET:TTST1GFSD4HKPE4VQAWJB3YVLDGZ2VIZTBI03BZQTXFI5FYA


In [29]:
LIMIT = 100 # limits number of returned venues

Create the user-defined function getNearbyVenues.

In [30]:
# will return up to 100 venues within 500m of each neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
# create a new dataframe of venues near Toronto neighborhoods
## (will print only neighborhood names when applying the function)
toronto_venues = getNearbyVenues(names = df_toronto['Neighborhood'],
                                   latitudes = df_toronto['Latitude'],
                                   longitudes = df_toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [32]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676845,-79.295225,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676845,-79.295225,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676845,-79.295225,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676845,-79.295225,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676845,-79.295225,Upper Beaches,43.680563,-79.292869,Neighborhood


In [33]:
print(toronto_venues.shape)

(1748, 7)


So 1748 venues are nearby Toronto neighborhoods.

In [34]:
# see the distribution of how many venues are near each neighborhood
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Adelaide, King, Richmond                                                                                      100
Berczy Park                                                                                                    64
Brockton, Exhibition Place, Parkdale Village                                                                   67
Business Reply Mail Processing Centre 969 Eastern                                                             100
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara     67
Cabbagetown, St. James Town                                                                                    39
Central Bay Street                                                                                             97
Chinatown, Grange Park, Kensington Market                                                                      95
Christie                                                                   

In [35]:
# determine the number of different/unique venue categories
print('There are {} unique venue categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 210 unique venue categories.


#### Create a *toronto_onehot* dataframe to show the distribution of venue categories

In [36]:
pd.set_option('display.max_columns', None) # so can see all of the columns (otherwise will see ... in the middle)

## create a new dataframe of dummy variables to see the venue category of venue near each neighborhood 
# perform one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the beginning
col_at_beg = ['Neighborhood']
toronto_onehot = toronto_onehot[[i for i in col_at_beg] 
        + [i for i in toronto_onehot if i not in col_at_beg]] 

toronto_onehot.head(10)

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [37]:
toronto_onehot.shape

(1748, 210)

#### Group by neighborhood and visualize the mean frequency per venue category

In [38]:
# use groupby to combine venues' data within the same neighborhood into one row each
# and show the mean frequency of venue category of venues within each neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.030000,0.00,0.010000,0.000000,0.000000,0.030000,0.0,0.000000,0.0,0.000000,0.020000,0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.0000,0.000000,0.000000,0.060000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.090000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.01,0.030000,0.00000,0.000000,0.01,0.000000,0.01,0.000000,0.010000,0.000000,0.030000,0.010000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.040000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.030000,0.000000,0.000000,0.0000

In [39]:
toronto_grouped.shape

(38, 210)

#### Determine the most-frequent venue categories per neighborhood and create a dataframe with this information

In [40]:
# determine the 5 most-frequent venue category types nearby each neighborhood
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n') # adds a blank line separating each neighborhood temp table

----Adelaide, King, Richmond----
          venue  freq
0   Coffee Shop  0.09
1          Café  0.06
2         Hotel  0.04
3     Gastropub  0.03
4  Burger Joint  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1    Restaurant  0.05
2  Cocktail Bar  0.05
3          Café  0.03
4        Bakery  0.03


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0     Coffee Shop  0.10
1            Café  0.06
2  Sandwich Place  0.04
3          Bakery  0.04
4      Restaurant  0.04


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.05
2   Steakhouse  0.04
3        Hotel  0.04
4          Bar  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0         Coffee Shop  0.10
1  Italian Restaurant  0.07
2                 Bar  0.04
3                Café  0.04
4          Restaurant  0.04


----Cabba

In [41]:
# create a user-defined function called return_most_common_venues
# will sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd'] # word part of 1st, 2nd, 3rd
columns = ['Neighborhood']

# create columns according to number of top venues
# for loop where column number increases by one and suffixes added as st, nd, rd then only with th
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) 
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe with the 10 most frequent nearby venues by category for each neighborhood
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Asian Restaurant,Bar,Steakhouse,Japanese Restaurant,Restaurant,Gym,Seafood Restaurant
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Seafood Restaurant,Bakery,Café,Breakfast Spot,Hotel,Cheese Shop,Italian Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Restaurant,Bakery,Furniture / Home Store,Sandwich Place,Vegetarian / Vegan Restaurant,Hotel,Pub,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Café,Bar,Steakhouse,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,Pizza Place,Sushi Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Italian Restaurant,Café,Bar,Restaurant,Sandwich Place,Speakeasy,Gym / Fitness Center,Park,Bakery


#### Use *k*-Means to create 5 neighborhood clusters, then visualize them on a map

In [43]:
from sklearn.cluster import KMeans # import k-means

kclusters = 5 # set number of clusters

# create a new dataframe and drop the Neighborhood column
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check the cluster labels/assignments for the first few rows
kmeans.labels_[0:20] 

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 0, 4, 3],
      dtype=int32)

In [44]:
# add clustering labels to neighborhoods_venues_sorted dataframe
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# create a new dataframe containing the same data as df_toronto
toronto_merged = df_toronto

# merge the df_toronto (now toronto_merged - see above line) and neighborhoods_venues_sorted dataframes into one - toronto_merged dataframe
# do so to have latitude and longitude values again
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676845,-79.295225,4,Health Food Store,Other Great Outdoors,Trail,Pub,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120,0,Park,Bus Line,Grocery Store,Discount Store,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314673,4,Park,Sandwich Place,Pizza Place,Fish & Chips Shop,Coffee Shop,Light Rail Station,Fast Food Restaurant,Burrito Place,Burger Joint,Food & Drink Shop
3,M4M,East Toronto,Studio District,43.662766,-79.334830,4,Bakery,Italian Restaurant,Diner,American Restaurant,Sushi Restaurant,Arts & Crafts Store,Coffee Shop,Brewery,Café,Bar
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085,2,Bus Line,Swim School,Yoga Studio,Elementary School,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


Visualize the clusters on a map.

In [47]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine each cluster more in depth

#### Cluster 0

In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"The Danforth West, Riverdale",0,Park,Bus Line,Grocery Store,Discount Store,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
5,Davisville North,0,Food & Drink Shop,Park,Breakfast Spot,Hotel,Gym,Clothing Store,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant
6,North Toronto West,0,Garden,Gym Pool,Playground,Park,Yoga Studio,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
8,"Moore Park, Summerhill East",0,Convenience Store,Playground,Tennis Court,Gym,Restaurant,Dance Studio,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
10,Rosedale,0,Bank,Park,Playground,Building,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
23,"Forest Hill North, Forest Hill West",0,Park,Lawyer,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
34,"High Park, The Junction South",0,Convenience Store,Park,Sandwich Place,Yoga Studio,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


Cluster 0 neighborhoods have more parks, playgrounds, and recreational areas nearby - a good fit for those who are active.

#### Cluster 1

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,1,Health & Beauty Service,Yoga Studio,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


The cluster 1 neighborhood has a health & beauty service nearby - a good fit for those who care for appearance.  
`Note: keep in mind, that the Roselawn neighborhood has only 1 venue within 500 meters.`  
Refer to the results of this earlier line of code: toronto_venues.groupby('Neighborhood')['Venue'].count()

#### Cluster 2

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,2,Bus Line,Swim School,Yoga Studio,Elementary School,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


The cluster 2 neighborhood has a bus line and swim school nearby - a good fit for a family with children.  
`Note: keep in mind, that the Lawrence Park neighborhood has only 2 venues within 500 meters.`  
Refer to the results of this earlier line of code: toronto_venues.groupby('Neighborhood')['Venue'].count()

#### Cluster 3

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,"Harbourfront East, Toronto Islands, Union Station",3,Harbor / Marina,Café,Music Venue,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


The cluster 3 neighborhood has a harbor/marina nearby and music venue - a good fit for those who like being near life near the marina.

#### Cluster 4

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,4,Health Food Store,Other Great Outdoors,Trail,Pub,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
2,"The Beaches West, India Bazaar",4,Park,Sandwich Place,Pizza Place,Fish & Chips Shop,Coffee Shop,Light Rail Station,Fast Food Restaurant,Burrito Place,Burger Joint,Food & Drink Shop
3,Studio District,4,Bakery,Italian Restaurant,Diner,American Restaurant,Sushi Restaurant,Arts & Crafts Store,Coffee Shop,Brewery,Café,Bar
7,Davisville,4,Dessert Shop,Pizza Place,Sandwich Place,Café,Italian Restaurant,Coffee Shop,Gym,Seafood Restaurant,Restaurant,Sushi Restaurant
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",4,Coffee Shop,Light Rail Station,Health & Beauty Service,Supermarket,Liquor Store,Gym,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
11,"Cabbagetown, St. James Town",4,Coffee Shop,Café,Pizza Place,Italian Restaurant,Restaurant,Bakery,Park,Farm,Japanese Restaurant,Jewelry Store
12,Church and Wellesley,4,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Dance Studio,Sushi Restaurant,Burger Joint,Men's Store,Hotel,Fast Food Restaurant
13,"Harbourfront, Regent Park",4,Coffee Shop,Gym / Fitness Center,Restaurant,Breakfast Spot,Bakery,Health Food Store,Italian Restaurant,Food Truck,Event Space,Mexican Restaurant
14,"Ryerson, Garden District",4,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Hotel,Diner,Plaza,Restaurant,Bookstore
15,St. James Town,4,Coffee Shop,Café,Restaurant,Hotel,Bakery,Seafood Restaurant,Italian Restaurant,Breakfast Spot,Gastropub,Clothing Store


Cluster 4 neighborhoods have more coffee shops, cafes, and food places nearby - a good fit for those who need a quick pick-me-up for work.

---

A peer-graded assignment for week 3 of IBM's Applied Data Science Capstone online course on Coursera.  
Notebook created by Paige Larsen. 

Acknowledgements: 
* Thank you to Corey Schafer's YouTube video, "Python Tutorial: Web Scraping with BeautifulSoup and Requests", for helping with BeautifulSoup
* Thank you to pythonprogramminglanguage.com's page on "Web Scraping with Pandas and Beautifulsoup", for helping to convert my soup into a *pandas* df
* Thank you to the various Stack Overflow pages consulted for ideas when I did not know how to progress
* Thank you to the various lessons and discussion forums of IBM's Data Science Professional Certificate courses on Coursera
* Thank you to Coursera's lab on Segmenting and Clustering Neighborhoods in New York City, for which most of Part 3 was based